In [1]:

# Import necessary libraries

from fastapi import FastAPI, File, UploadFile
import uvicorn
import numpy as np
import cv2
from io import BytesIO
from fastapi.responses import StreamingResponse
import nest_asyncio  

In [2]:

# Initialize FastAPI app

app = FastAPI()



In [3]:
# Function to detect apples in an image using color pallets and contours 

def detect_apples(image: np.ndarray):
    # Convert image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define color range for red apples
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])
    
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = mask1 + mask2
    
    # Apply morphological operations to remove noise
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw bounding boxes around detected apples
    for contour in contours:
        if cv2.contourArea(contour) > 1000:  # Filter small detections
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 3)
    
    return image

In [4]:
# API endpoint to upload images and detect if they are apples or not 
@app.post("/is_apple")
async def is_apple(file: UploadFile = File(...)):
    try:
        # Debug: Print file details
        print(f"Received file: {file.filename}, Content Type: {file.content_type}")

        # Read image file
        image_bytes = await file.read()
        
        # Convert to NumPy array
        image_array = np.frombuffer(image_bytes, np.uint8)
        
        # Decode image
        image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
        
        if image is None:
            print("Error: Could not decode image")
            return {"error": "Invalid image file"}

        # Process image
        processed_image = detect_apples(image)

        # Encode image
        _, encoded_image = cv2.imencode(".jpg", processed_image)
        
        return StreamingResponse(BytesIO(encoded_image.tobytes()), media_type="image/jpeg")

    except Exception as e:
        print(f"Error: {e}")
        return {"error": str(e)}

In [5]:
# Runnin the API server
nest_asyncio.apply()  # Allows FastAPI to run inside Jupyter

uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [62250]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51340 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:51340 - "GET /openapi.json HTTP/1.1" 200 OK
Received file: download.jpeg, Content Type: image/jpeg
INFO:     127.0.0.1:51344 - "POST /is_apple HTTP/1.1" 200 OK
Received file: REDDELAPP6_1400x.webp, Content Type: image/webp
INFO:     127.0.0.1:51350 - "POST /is_apple HTTP/1.1" 200 OK
Received file: apple-2720105_1280.png, Content Type: image/png
INFO:     127.0.0.1:51352 - "POST /is_apple HTTP/1.1" 200 OK
Received file: download.jpeg, Content Type: image/jpeg
INFO:     127.0.0.1:51354 - "POST /is_apple HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [62250]


references for the code used: 
National Film Board of Canada. (2017, April 12). Trick or Treaty? - Alanis Obomsawin (2014) [HD, 52 min] [Video]. YouTube. https://www.youtube.com/watch?v=tc5-wp7D9ko 
most of image detection video served as references point for my code
